# Tarea 1: Búsqueda de Caserones



La tarea tiene como objetivos crear un modelo de caserones y una lista de caserones en conflictos, a partir de un modelo de recursos y dimensiones dadas en un archivo .csv llamado Tarea_1.


Primero, se importan las librerías que se usarán en el trabajo (pandas y numpy).

In [1]:
import pandas as pd
import numpy as np

Luego se llama al archivo Tarea_1 de la carpeta de donde se encuentre (si usted la tiene en una carpeta distinta, cambie la dirección del código), y se guarda como una tabla de pandas DataFrame, que facilita y optimiza su modificación.

In [2]:
modelo_bloque = pd.read_csv(r"C:\Users\jorge\Documents\jorge U\Planificación minera\Tarea_1.csv")
#print(modelo_bloque)

Se agregan 2 columnas correspondientes al tonelaje de finos de cobre y oro (en toneladas), y que nos servirán posteriormente para calcular la ley de los caserones.

In [3]:
modelo_bloque["Finos Cu [t]"] = modelo_bloque["cu[%]"] * modelo_bloque["tons [t]"] / 100
modelo_bloque["Finos Au [t]"] = modelo_bloque["au[ppm]"] * modelo_bloque["tons [t]"] / 1000000

print(modelo_bloque)

         X    Y   Z  sg[t/m3]  tons [t]  au[ppm]  cu[%]  profit [$]  \
0        0    0   0      2.27    283.33      0.0    0.0    -7083.33   
1        0    0   5      2.27    283.33      0.0    0.0    -7083.33   
2        0    0  10      2.27    283.33      0.0    0.0    -7083.33   
3        0    0  15      2.27    283.33      0.0    0.0    -7083.33   
4        0    0  20      2.27    283.33      0.0    0.0    -7083.33   
...    ...  ...  ..       ...       ...      ...    ...         ...   
53266  300  295  60      2.27    283.33      0.0    0.0    -7083.33   
53267  300  295  65      2.27    283.33      0.0    0.0    -7083.33   
53268  300  295  70      2.27    283.33      0.0    0.0    -7083.33   
53269  300  295  75      2.27    283.33      0.0    0.0    -7083.33   
53270  300  295  80      2.27    283.33      0.0    0.0    -7083.33   

       Finos Cu [t]  Finos Au [t]  
0               0.0           0.0  
1               0.0           0.0  
2               0.0           0.0  
3  

#### Se crea la tabla de caserones, el entregable, inicialmente como una tabla vacía, con los siguientes atrubutos:

    -X_i, Y_i, Z_i : Coordenadas iniciales del caserón (primero bloque abajo, a la izquiera y al fondo).

    -X_f, Y_f, Z_f : Coordenadas finales del caserón (Ultimo bloque arriba, a la derecha y al frente).

    -Cu [%] : Ley de cobre en porcentaje del caserón.

    -Au [ppm] : Ley de oro en ppm del caserón.

    -Ton [t] : Tonelaje del caserón.

    -B [$] : Beneficio en pesos de extraer el caserón.

In [4]:
caserones = pd.DataFrame({"X_i":[], "Y_i":[], "Z_i":[],
                     "X_f":[], "Y_f":[], "Z_f":[],
                     "Cu [%]":[], "Au [ppm]":[], "Ton [t]":[], "B [$]":[]})
print(caserones)

Empty DataFrame
Columns: [X_i, Y_i, Z_i, X_f, Y_f, Z_f, Cu [%], Au [ppm], Ton [t], B [$]]
Index: []


## Caserones

A continuación, se crea una excel con todos los caserones posibles del modelo de bloques.

In [8]:
n_bloques = len(modelo_bloque.index)
contador = 0

dx = 20          #dimension x del caseron
dy = 15          #dimension y del caseron
dz = 30          #dimension z del caseron

bx = 5           #dimension x del Bloque
by = 5           #dimension y del Bloque
bz = 5           #dimension z del Bloque

n_bloques_x = dx / bx
n_bloques_y = dy / by
n_bloques_z = dz / bz

cantidad_bloques_caseron = n_bloques_x * n_bloques_y * n_bloques_z   # Nos entrega la cantidad de bloque que debe haber
                                                                     # en un caserón, hay caserones que se generan con menos 
                                                                     # bloques, dado que son cortados por la topografía, 
                                                                     # estos no se consideran,

bloques_caseron_x = dx/bx  #cantidad de bloques en x que caben en un caseron
bloques_caseron_y = dy/by  #cantidad de bloques en y que caben en un caseron
bloques_caseron_z = dz/bz  #cantidad de bloques en z que caben en un caseron

mx = modelo_bloque["X"].max()  #Máxima coordenada en x del modelo entregado
my = modelo_bloque["Y"].max()  #Máxima coordenada en y del modelo entregado
mz = modelo_bloque["Z"].max()  #Máxima coordenada en z del modelo entregado

for i in range(n_bloques):
    cx_i = modelo_bloque.loc[i,"X"]  #Coordenada inicial en x del caseron (limite inferior)
    cy_i = modelo_bloque.loc[i,"Y"]  #Coordenada inicial en y del caseron (limite inferior)
    cz_i = modelo_bloque.loc[i,"Z"]  #Coordenada inicial en z del caseron (limite inferior)
    
    cx_f = cx_i + dx - bx            #Coordenada final en x del caseron (limite superior)
    cy_f = cy_i + dy - by            #Coordenada final en y del caseron (limite superior)
    cz_f = cz_i + dz - bz            #Coordenada final en z del caseron (limite superior)
    
    if (cx_f  <= mx) and (cy_f <= my) and (cz_f <= mz):   # Esta restricción permite que no se tomen caserones iniciales 
                                                          # cuyas coordenadas finales se encuentran fuera del modelo de bloques
            
        #Se filtran los bloques que exceden los limites establecidos de las dimensiones de los caserones
        bloques_caseron = modelo_bloque.loc[ (modelo_bloque["X"] >= cx_i) & (modelo_bloque["X"] <= cx_f) &
                                            (modelo_bloque["Y"] >= cy_i) & (modelo_bloque["Y"] <= cy_f) &
                                            (modelo_bloque["Z"] >= cz_i) & (modelo_bloque["Z"] <= cz_f) ]
        
        n_b = len(bloques_caseron.index)                #La cantidad de bloques por caseron debe ser igual a 72 para este caso 
        b = float(bloques_caseron["profit [$]"].sum())  #Beneficio total del caseron (suma de beneficio de bloques)

        if b >= 0 and n_b == cantidad_bloques_caseron:  #Se buscan solo los que tengan beneficios positivos y 72 bloques
            
            #Se calcula tonelaje total del caseron
            ton = float(bloques_caseron["tons [t]"].sum())
            
            #Se calcular la ley de cobre de cada caserón
            finos_cu = float(bloques_caseron["Finos Cu [t]"].sum())
            ley_cu = 100 * finos_cu / ton
            
            #Se calcula la ley de oro de cada caserón
            finos_au = float(bloques_caseron["Finos Au [t]"].sum())
            ley_au = 1000000 * finos_au / ton 
            
            #Se agregan los datos a la tabla de caserones
            caserones.loc[contador] = [cx_i, cy_i, cz_i, cx_f, cy_f, cz_f,
                                      ley_cu, ley_au, ton, b]
            contador = contador + 1

print(caserones)
print("El código arroja un total de", contador, "caserones.")

        X_i    Y_i   Z_i    X_f    Y_f   Z_f    Cu [%]  Au [ppm]   Ton [t]  \
0      45.0  100.0  20.0   60.0  110.0  45.0  0.347072  0.082996  20877.62   
1      45.0  100.0  25.0   60.0  110.0  50.0  0.415583  0.102596  21001.02   
2      45.0  100.0  30.0   60.0  110.0  55.0  0.441991  0.111409  21060.53   
3      45.0  100.0  35.0   60.0  110.0  60.0  0.415248  0.107294  21047.53   
4      45.0  100.0  40.0   60.0  110.0  65.0  0.326537  0.088607  20942.33   
...     ...    ...   ...    ...    ...   ...       ...       ...       ...   
5603  185.0  120.0  20.0  200.0  130.0  45.0  0.563327  0.077422  20855.49   
5604  185.0  125.0   0.0  200.0  135.0  25.0  0.412948  0.055185  20725.91   
5605  185.0  125.0   5.0  200.0  135.0  30.0  0.441209  0.059121  20749.99   
5606  185.0  125.0  10.0  200.0  135.0  35.0  0.441209  0.059121  20749.99   
5607  185.0  125.0  15.0  200.0  135.0  40.0  0.433670  0.058023  20744.80   

          B [$]  
0      53831.95  
1     167194.45  
2     212

#### Se procede a guardar el modelo en un archivo de nombre "Modelo de caserones".

(Si desea guardarlo en una carpeta distinta, modifique la dirección en el código)

In [7]:
caserones.to_csv(r"C:\Users\jorge\Documents\jorge U\Planificación minera\Modelo de caserones.csv", index=True)

## Caserones en conflicto

A continuación, en la tabla de caserones creada anteriormente agregamos una columna de listas de python, las cuales van a contener a los caserones que traslapan con cada uno de los ya identificados, según el índice que tienen en la tabla creada.

In [9]:
caserones["Conflicto"] = 0
caserones["Conflicto"] = caserones["Conflicto"].astype(object)

n = len(caserones.index)
for i in range(n):
    caserones.at[i,"Conflicto"] = []
    
print(caserones)

        X_i    Y_i   Z_i    X_f    Y_f   Z_f    Cu [%]  Au [ppm]   Ton [t]  \
0      45.0  100.0  20.0   60.0  110.0  45.0  0.347072  0.082996  20877.62   
1      45.0  100.0  25.0   60.0  110.0  50.0  0.415583  0.102596  21001.02   
2      45.0  100.0  30.0   60.0  110.0  55.0  0.441991  0.111409  21060.53   
3      45.0  100.0  35.0   60.0  110.0  60.0  0.415248  0.107294  21047.53   
4      45.0  100.0  40.0   60.0  110.0  65.0  0.326537  0.088607  20942.33   
...     ...    ...   ...    ...    ...   ...       ...       ...       ...   
5603  185.0  120.0  20.0  200.0  130.0  45.0  0.563327  0.077422  20855.49   
5604  185.0  125.0   0.0  200.0  135.0  25.0  0.412948  0.055185  20725.91   
5605  185.0  125.0   5.0  200.0  135.0  30.0  0.441209  0.059121  20749.99   
5606  185.0  125.0  10.0  200.0  135.0  35.0  0.441209  0.059121  20749.99   
5607  185.0  125.0  15.0  200.0  135.0  40.0  0.433670  0.058023  20744.80   

          B [$] Conflicto  
0      53831.95        []  
1     1

### Incompatibilidad por pilar o losa

Para encontrar los casrones que traslapan, se buscan todos aquellos bloques que, por la posición de su coordenada inicial, compartirían un espacio con el caseron analizado, indistintamente de si traslapa por losa o pilar. Se filtran aquellos bloques que trasapan mediante la siguientes restricciónes:

    1) La coordenada inicial de x debe ser mayor que la coordenada incial en x del bloque analizado menos el ancho del      caseron, y menor a la coordenada inicial del bloque analizado mas el ancho del caseron (Restricción por pilar).

    2) La coordenada inicial de y debe ser mayor que la coordenada incial en y del bloque analizado menos el largo del      caseron, y menor a la coordenada inicial del bloque analizado mas el largo del caseron (Restricción por pilar).

    3) La coordenada inicial de z debe ser mayor que la coordenada incial en z del bloque analizado menos el alto del        caseron, y menor a la coordenada inicial del bloque analizado mas el alto del caseron (Restricción por losa).
    
Las tres restricciones no son independientes y deben cumplirse todas para que el caserón traslape, dichas restricciones nos entregarán todos los bloques que traslapen, sin embargo incluirán el bloque mismo que se está analizando, por lo cual, se encuentra el índice del bloque y se elimina de la tabla.

In [10]:
for i in range(n):
    cxi = caserones.loc[i,"X_i"]  #Coordenada x inicial del caserón analizado
    cyi = caserones.loc[i,"Y_i"]  #Coordenada y inicial del caserón analizado
    czi = caserones.loc[i,"Z_i"]  #Coordenada z inicial del caserón analizado
    incompatibles = caserones.loc[ (caserones["X_i"] > cxi - dx) & (caserones["X_i"] < cxi + dx) &   #Restricción (1) (pilar)
                                  (caserones["Y_i"] > cyi - dy) & (caserones["Y_i"] < cyi + dy) &    #Restricción (2) (pilar)
                                  (caserones["Z_i"] > czi - dz) & (caserones["Z_i"] < czi + dz) ]    #Restricción (3) (losa)
                                
    #Para evitar que un caserón aparezca que traslapa consigo mismo
    caseron_dado = caserones[(caserones["X_i"] == cxi) & (caserones["Y_i"] == cyi) & (caserones["Z_i"] == czi)].index
    cd = caseron_dado[0]                     #Se guarda el índice del caserón analizado
    incompatibles = incompatibles.drop(cd)   #Se elimina de la tabla de incompatibles
    
    lista_incompatibles = list(incompatibles.index)
    caserones.loc[i,"Conflicto"][:]=lista_incompatibles

print(caserones)

        X_i    Y_i   Z_i    X_f    Y_f   Z_f    Cu [%]  Au [ppm]   Ton [t]  \
0      45.0  100.0  20.0   60.0  110.0  45.0  0.347072  0.082996  20877.62   
1      45.0  100.0  25.0   60.0  110.0  50.0  0.415583  0.102596  21001.02   
2      45.0  100.0  30.0   60.0  110.0  55.0  0.441991  0.111409  21060.53   
3      45.0  100.0  35.0   60.0  110.0  60.0  0.415248  0.107294  21047.53   
4      45.0  100.0  40.0   60.0  110.0  65.0  0.326537  0.088607  20942.33   
...     ...    ...   ...    ...    ...   ...       ...       ...       ...   
5603  185.0  120.0  20.0  200.0  130.0  45.0  0.563327  0.077422  20855.49   
5604  185.0  125.0   0.0  200.0  135.0  25.0  0.412948  0.055185  20725.91   
5605  185.0  125.0   5.0  200.0  135.0  30.0  0.441209  0.059121  20749.99   
5606  185.0  125.0  10.0  200.0  135.0  35.0  0.441209  0.059121  20749.99   
5607  185.0  125.0  15.0  200.0  135.0  40.0  0.433670  0.058023  20744.80   

          B [$]                                          Confli

#### Armar pares de caserones que traslapan

Se extrae la columna de listas con los caserones en conflicto y se emparejan con su indice, el cual corresponde tambien al número del caserón con el que traslapa.

In [11]:
lista_conflictivos = caserones["Conflicto"] #Se extrae la comulna de caserones en conflicto
largo_lista = len(lista_conflictivos)       #Se calcula el tamaño de dicha lista

#Se ordena como pares ordenados
pares_conflictivos = [ [a, b]
                      for a in range(largo_lista)
                      for b in lista_conflictivos[a]]

l = len(pares_conflictivos)
print("Se encuentran", l, "pares de caserones conflictivos, sin filtrar.")

pares = pd.DataFrame(pares_conflictivos)

Se encuentran 1355600 pares de caserones conflictivos, sin filtrar.


Muchos de los pares se repiten, vale decir (a,b) y (b,a), por lo cual se deben flitrar dichos casos.

Para ello se usa la función sort de numpy para ordenarlos de manera que quede el menor primero, y luego se eliminan los pares duplicados (tal como se vió en auxiliar).

In [12]:
duplicados = pd.DataFrame(np.sort(pares[[0,1]], axis=1), index=pares.index).duplicated()
pares = pares[~duplicados]
print(pares)
h = len(pares)
print("Se encuentran", h, "pares de caserones en conflicto")

            0     1
0           0     1
1           0     2
2           0     3
3           0     4
4           0     5
...       ...   ...
1355280  5604  5606
1355281  5604  5607
1355377  5605  5606
1355378  5605  5607
1355484  5606  5607

[677800 rows x 2 columns]
Se encuentran 677800 pares de caserones en conflicto


### Exportamos la tabla

Se exporta a un archivo llamado "Caserones traslapados" (Si desea guardarlo en otra parte, cambie dirección en el código).

In [13]:
pares.to_csv(r"C:\Users\jorge\Documents\jorge U\Planificación minera\Caserones trasplapados.csv",index=False)